<a href="https://colab.research.google.com/github/rubenguerra/datascience/blob/master/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Usando una red GAN para generar imágenes realistas desde el Cifar10 dataset

In [1]:
from numpy import expand_dims
from numpy import zeros, ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import  Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from matplotlib import pyplot

#Discriminador

In [2]:
def define_discriminator(in_shape=(32, 32, 3)):
  model=Sequential()
  model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(128,(3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [3]:
def define_generator(latent_dim):
  model=Sequential()
  n_nodes=256*4*4
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((4,4,256)))
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
  return model

#Definimos la Red GAN

In [4]:
def define_gan(g_model, d_model):
  d_model.trainable = False
  model=Sequential()
  model.add(g_model)
  model.add(d_model)
  opt=Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

#Configuracion de datos de entrada

In [5]:
def load_real_samples():
  (trainX, _), (_,_) = load_data()
  X=trainX.astype('float32')
  X=(X - 127.5) / 127.5
  return X

def generate_real_samples(dataset, n_samples):
  ix=randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))
  return X, y

def generate_latent_points(latent_dim, n_samples):
  x_input=randn(latent_dim * n_samples)
  x_input=x_input.reshape(n_samples, latent_dim)
  return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input=generate_latent_points(latent_dim, n_samples)
  X= g_model.predict(x_input)
  y = zeros((n_samples, 1))
  return X, y
  

#Entrenando la neurona GAN

In [8]:
def train(g_model, d_model,gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
  bat_per_epo=int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model,dataset, latent_dim)

In [9]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
  X_real, y_real=generate_real_samples(dataset, n_samples)
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  x_fake, y_fake=generate_fake_samples(g_model, latent_dim, n_samples)
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
  save_plot(x_fake, epoch)
  filename='generator_model_%03d.h5' % (epoch+1)
  g_model.save(filename)

#Ejecucion

In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset=load_real_samples()
train(g_model, d_model, gan_model, dataset,latent_dim)

170500096/170498071 [==============================] - 2s 0us/step
>1, 1/390, d1=0.700, d2=0.696 g=0.691
>1, 2/390, d1=0.638, d2=0.698 g=0.690
>1, 3/390, d1=0.583, d2=0.702 g=0.686
>1, 4/390, d1=0.505, d2=0.711 g=0.678
>1, 5/390, d1=0.375, d2=0.729 g=0.667
>1, 6/390, d1=0.259, d2=0.752 g=0.662
>1, 7/390, d1=0.180, d2=0.761 g=0.678
>1, 8/390, d1=0.098, d2=0.730 g=0.719
>1, 9/390, d1=0.075, d2=0.696 g=0.766
>1, 10/390, d1=0.052, d2=0.693 g=0.803
>1, 11/390, d1=0.039, d2=0.684 g=0.838
>1, 12/390, d1=0.060, d2=0.659 g=0.861
>1, 13/390, d1=0.043, d2=0.646 g=0.883
>1, 14/390, d1=0.058, d2=0.628 g=0.886
>1, 15/390, d1=0.097, d2=0.652 g=0.854
>1, 16/390, d1=0.069, d2=0.696 g=0.841
>1, 17/390, d1=0.095, d2=0.805 g=0.802
>1, 18/390, d1=0.081, d2=0.885 g=0.789
>1, 19/390, d1=0.077, d2=0.839 g=0.891
>1, 20/390, d1=0.131, d2=0.636 g=1.050
>1, 21/390, d1=0.079, d2=0.496 g=1.153
>1, 22/390, d1=0.043, d2=0.454 g=1.196
>1, 23/390, d1=0.052, d2=0.444 g=1.179
>1, 24/390, d1=0.039, d2=0.442 g=1.140
>1, 25